In [ ]:
import base64
import json
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

### Session

In [ ]:
import boto3, time, json

sess = boto3.Session()
sm = sess.client("sagemaker")
region = sess.region_name
account = boto3.client("sts").get_caller_identity().get("Account")

In [ ]:
pip install sagemaker

In [ ]:
!wget https://s3.us-west-2.amazonaws.com/mar-files-ts0.4/flores_small.mar

### IAM Role

Note: make sure the IAM role has:

AmazonS3FullAccess

AmazonEC2ContainerRegistryFullAccess

AmazonSageMakerFullAccess

In [ ]:
import sagemaker

# role = sagemaker.get_execution_role()
role = 'arn:aws:iam::320567679581:role/Hamid'

### Amazon Elastic Container Registry (ECR)

In [ ]:
registry_name = "dynalab-torchserve-sagemaker"

In [ ]:
image = f"{account}.dkr.ecr.{region}.amazonaws.com/{registry_name}:latest"


In [ ]:
image

In [ ]:
# you don't need to run the next cell as if it has already been registered

In [ ]:
!aws ecr create-repository --repository-name {registry_name}

### Pytorch Model Artifact (mar file)

In [ ]:
model_file_name = "flores_small"
sagemaker_session = sagemaker.Session()
bucket_name = sagemaker_session.default_bucket()
prefix = 'Dyna'

No need to run next cell as if it has been already pushed to s3 bucket

In [ ]:
!tar cvfz {model_file_name}.tar.gz flores_small.mar
!aws s3 cp {model_file_name}.tar.gz s3://{bucket_name}/{prefix}/models/

The model_artifact is equal to the file uploaded in the S3 bucket as will be printed when the above cell ran. 

In [ ]:
model_artifact = 's3://sagemaker-us-west-2-320567679581/Dyna/models/flores_small.tar.gz' # This should be changed to S3 path generated above



### Build a Customized TorchServe Docker container and push it to Amazon ECR

In [ ]:
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account}.dkr.ecr.{region}.amazonaws.com


In [ ]:
!docker build -t {registry_name} /home/ubuntu/serve/examples/Sagemaker

In [ ]:
!docker tag {registry_name}:latest {image}

In [ ]:
!docker push {image}

In [ ]:
model_name = "floressmall-torchserve-sagemaker"

### Create a Model for Sagemaker use

In [ ]:
container = {"Image": image, "ModelDataUrl": model_artifact}

create_model_response = sm.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=container
)
print(create_model_response["ModelArn"])


### Batch transform jobs

* The s3_bucket_name is the bucket_name that has been created at the start of the notebook.
* Make sure in the bucker name you create the batch_input and batch_output folders as shown below.
* Make sure the dataset files/ shared input files, are placed in the batch_input folder.

In [ ]:
s3_bucket_name= 'sagemaker-us-west-2-320567679581'
batch_input = f"s3://{s3_bucket_name}/Dyna/batch_transform_flores_torchserve_sagemaker/"
batch_output = f"s3://{s3_bucket_name}/Dyna/batch_transform_flores_torchserve_sagemaker_output/"

In [ ]:
import time

batch_job_name = 'flores-batch' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
batch_job_name

In [ ]:
request = {
    "ModelClientConfig": {
        "InvocationsTimeoutInSeconds": 3600,
        "InvocationsMaxRetries": 1,
    },
    "TransformJobName": batch_job_name,
    "ModelName": model_name,
    "MaxConcurrentTransforms":1,
    "BatchStrategy": "MultiRecord",
    "TransformOutput": {"S3OutputPath": batch_output, "AssembleWith": "Line", "Accept": "application/json"},
    "TransformInput": {
        "DataSource": {
            "S3DataSource": {"S3DataType": "S3Prefix", "S3Uri": batch_input}
        },
        
        "SplitType" : "Line",
        "ContentType": "application/json",
    },
    "TransformResources": {"InstanceType": "ml.p2.xlarge", "InstanceCount": 1},
}


In [ ]:
%%time
sm.create_transform_job(**request)

while True:
    response = sm.describe_transform_job(TransformJobName=batch_job_name)
    status = response["TransformJobStatus"]
    if status == "Completed":
        print("Transform job ended with status: " + status)
        break
    if status == "Failed":
        message = response["FailureReason"]
        print("Transform failed with the following error: {}".format(message))
        raise Exception("Transform job failed")
    print("Transform job is still in status: " + status)
    time.sleep(30)

#### Batch aggregated requests

In [ ]:
client = boto3.client("sagemaker")
client.delete_model(ModelName=model_name)
